# Tensor Factorization for Recommender Systems
### - CP factorization on tensor (user, music, context)
### - Written by ByungSoo Jeon, NAVER LABS

In [28]:
# Add system path to use scikit-tensor Library
import sys
sys.path.append('/Users/jbsimdicd/Library/Python/2.7/lib/python/site-packages')

from scipy.io.matlab import loadmat
from sktensor import dtensor, cp_als
import numpy as np

# Set logging to DEBUG to see CP-ALS information
import logging
logging.basicConfig(level=logging.DEBUG)

### - Naive CP tensor factorization 

In [29]:
# # Load Matlab data
# mat = loadmat('../data/sensory-bread/brod.mat')
# # Create dense tensor from numpy array
# T = dtensor()
# # Decompose tensor using CP-ALS
# P, fit, itr, exectimes = cp_als(T, 3, init='random')

### - Regularized CP tensor factorization (Multiverse Recommendation, RecSys 10)

In [109]:
def Regularized_CP_TF(X, A, B, C, R, steps=5000, alpha=0.0002, beta=0.02):
    e = 0
    for step in range(steps):
        # Stochastic Gradient Descent(SGD) part
        for i in range(len(X)):
            for j in range(len(X[i])):
                for k in range(len(X[i][j])):
                    if X[i][j][k] > 0:
                        eijk = X[i][j][k] - np.dot(A[i,:],np.multiply(B[j,:],C[k,:]))
                        for r in range(R):
                            A[i][r] = A[i][r] + alpha * (2 * eijk * B[j][r] * C[k][r] - beta * A[i][r])
                            B[j][r] = B[j][r] + alpha * (2 * eijk * A[i][r] * C[k][r] - beta * B[j][r])
                            C[k][r] = C[k][r] + alpha * (2 * eijk * B[j][r] * A[i][r] - beta * C[k][r])                            
        
        # Stop condition for SGD
        e = 0
        for i in range(len(X)):
            for j in range(len(X[i])):
                for k in range(len(X[i][j])):
                    if X[i][j][k] > 0:
                        e = e + pow(X[i][j][k] - np.dot(A[i,:],np.multiply(B[j,:],C[k,:])), 2)
#                         for r in range(R):
#                             e = e + (beta/2) * (pow(A[i][r],2) + pow(B[j][r],2) + pow(C[k][r],2))
        if e < 0.001:
            break
    return A,B,C,e

### - Regularized Tucker tensor factorization (a.k.a HOSVD)

In [121]:
def Regularized_Tucker_TF(X, A, B, C, G, P, Q, R, steps=20000, alpha=0.0002, beta=0.02):
    e = 0
    for step in range(steps):
        # Stochastic Gradient Descent(SGD) part
        for i in range(len(X)):
            for j in range(len(X[i])):
                for k in range(len(X[i][j])):
                    if X[i][j][k] > 0:
                        sum = 0
                        for p in range(P):
                            for q in range(Q):
                                for r in range(R):
                                    sum += G[p][q][r]*A[i][p]*B[j][q]*C[k][r]
                        eijk = X[i][j][k] - sum
                        
                        for p in range(P):
                            sum = 0
                            for q in range(Q):
                                for r in range(R):
                                    sum += B[j][q]*C[k][r]*G[p][q][r]
                            A[i][p] = A[i][p] + alpha * (2 * eijk * sum - beta * A[i][p])
                        for q in range(Q):
                            sum = 0
                            for p in range(P):
                                for r in range(R):
                                    sum += A[i][p]*C[k][r]*G[p][q][r]
                            B[j][q] = B[j][q] + alpha * (2 * eijk * sum - beta * B[j][q])
                        for r in range(R):
                            sum = 0
                            for q in range(Q):
                                for p in range(P):
                                    sum += B[j][q]*A[i][p]*G[p][q][r]
                            C[k][r] = C[k][r] + alpha * (2 * eijk * sum - beta * C[k][r])        
                        for r in range(R):
                            for q in range(Q):
                                for p in range(P):
                                    G[p][q][r] = G[p][q][r] + alpha*(2*A[i][p]*B[j][q]*C[k][r]-beta*G[p][q][r])
        
        # Stop condition for SGD
        e = 0
        for i in range(len(X)):
            for j in range(len(X[i])):
                for k in range(len(X[i][j])):
                    if X[i][j][k] > 0:
                        sum = 0
                        for p in range(P):
                            for q in range(Q):
                                for r in range(R):
                                    sum += G[p][q][r]*A[i][p]*B[j][q]*C[k][r]

                        e = e + pow(X[i][j][k] - sum, 2)
                        
#                         for p in range(P):
#                             e = e + (beta/2) * pow(A[i][p],2)
#                         for q in range(Q):
#                             e = e + (beta/2) * pow(B[j][q],2)
#                         for p in range(R):
#                             e = e + (beta/2) * pow(C[k][r],2)

        if e < 0.001:
            break
    return A,B,C,G,e

### - Main function for CP

In [122]:
if __name__ == "__main__":
    X = [1,2,3,4,5,6,7,8,9,10,11,12]
    X = np.array(X)
    X = X.reshape(2,2,3)
    R = 2

    A = np.random.rand(len(X),R)
    B = np.random.rand(len(X[0]),R)
    C = np.random.rand(len(X[0][0]),R)
    
    A, B, C, e = Regularized_TF(X, A, B, C, R)
    print (e)

1.97233010163


### - Main function for Tucker

In [123]:
if __name__ == "__main__":
    X = [1,2,3,4,5,6,7,8,9,10,11,12]
    X = np.array(X)
    X = X.reshape(2,2,3)
    P, Q, R = 2, 2, 2

    A = np.random.rand(len(X),P)
    B = np.random.rand(len(X[0]),Q)
    C = np.random.rand(len(X[0][0]),R)
    G = np.random.rand(P,Q,R)
    
    A,B,C,G,e = Regularized_Tucker_TF(X,A,B,C,G,P,Q,R)
    print (G)
    print (e)

/Users/jbsimdicd/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:51: RuntimeWarning: overflow encountered in double_scalars
/Users/jbsimdicd/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:49: RuntimeWarning: overflow encountered in double_scalars
/Users/jbsimdicd/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: RuntimeWarning: overflow encountered in double_scalars
/Users/jbsimdicd/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in double_scalars


[[[ nan  nan]
  [ nan  nan]]

 [[ nan  nan]
  [ nan  nan]]]
nan
